In [1]:
import pyspark
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/19 14:07:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql import Row

In [4]:
data = [
    Row(delivery_id=1, customer_id=1, order_date="2019-08-01", customer_pref_delivery_date="2019-08-02"),
    Row(delivery_id=2, customer_id=5, order_date="2019-08-02", customer_pref_delivery_date="2019-08-02"),
    Row(delivery_id=3, customer_id=1, order_date="2019-08-11", customer_pref_delivery_date="2019-08-11"),
    Row(delivery_id=4, customer_id=3, order_date="2019-08-24", customer_pref_delivery_date="2019-08-26"),
    Row(delivery_id=5, customer_id=4, order_date="2019-08-21", customer_pref_delivery_date="2019-08-22"),
    Row(delivery_id=6, customer_id=2, order_date="2019-08-11", customer_pref_delivery_date="2019-08-13")
]
spark = SparkSession.builder.appName("delivery_data").getOrCreate()

# Convert the list of Row objects into a DataFrame
df = spark.createDataFrame(data)
df.show()

23/12/19 14:07:59 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----------+-----------+----------+---------------------------+
|delivery_id|customer_id|order_date|customer_pref_delivery_date|
+-----------+-----------+----------+---------------------------+
|          1|          1|2019-08-01|                 2019-08-02|
|          2|          5|2019-08-02|                 2019-08-02|
|          3|          1|2019-08-11|                 2019-08-11|
|          4|          3|2019-08-24|                 2019-08-26|
|          5|          4|2019-08-21|                 2019-08-22|
|          6|          2|2019-08-11|                 2019-08-13|
+-----------+-----------+----------+---------------------------+



In [5]:
df.createOrReplaceTempView("Delivery")

23/12/19 14:08:10 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [9]:
result = spark.sql(
    """
    select round(100*count(if(order_date = customer_pref_delivery_date, 1, null))/count(*),2) as immediate_percentage
from Delivery
    """
)
result.show()

+--------------------+
|immediate_percentage|
+--------------------+
|               33.33|
+--------------------+



In [14]:
result = spark.sql(
    """
    SELECT ROUND(100 * SUM(CASE WHEN order_date = customer_pref_delivery_date THEN 1 ELSE 0 END) / COUNT(*), 2) AS immediate_percentage
FROM Delivery;


    """
)
result.show()

+--------------------+
|immediate_percentage|
+--------------------+
|               33.33|
+--------------------+



In [14]:
result = spark.sql(
    """
    SELECT IFNULL(
        ROUND((SELECT COUNT(delivery_id) FROM Delivery WHERE order_date = customer_pref_delivery_date)
            /(SELECT COUNT(*) FROM Delivery) * 100, 2)
        , 0) AS immediate_percentage
    """
)
result.show()

+--------------------+
|immediate_percentage|
+--------------------+
|               33.33|
+--------------------+

